# Makina Öğrenmesi Modelleri İle Sınıflandırma

In [144]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy, textblob, string, xgboost
from warnings import filterwarnings
filterwarnings('ignore')
import pandas as pd 
import numpy as np
import time
import TweetAnaliz as tw

In [145]:
data = pd.read_csv('yorumlar.csv', encoding ='utf_8', sep='"')
data['Sentiment'] = 1
data.Sentiment.iloc[10003:] = 0

In [146]:
data["Sentiment"].replace(0, value = "negatif", inplace = True)
data["Sentiment"].replace(1, value = "pozitif", inplace = True)

In [147]:
data.columns

Index(['yorum,Positivity', 'Sentiment'], dtype='object')

In [148]:
data.rename(columns ={'yorum,Positivity':'yorum'},inplace=True)

In [149]:
data = data[(data.Sentiment == "negatif") | (data.Sentiment == "pozitif")]

In [150]:
data.groupby("Sentiment").count()

,yorum
Sentiment,
negatif,9996
pozitif,10003


In [151]:
df = pd.DataFrame()
df["text"] = data["yorum"]
df["label"] = data["Sentiment"]

In [152]:
df.drop('id', 1)    İd satırını siler

SyntaxError: invalid syntax (<ipython-input-152-98c4109d9c61>, line 1)

# Test Train

In [153]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df["text"],df["label"])

In [154]:
encoder = preprocessing.LabelEncoder()

In [155]:
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

# Değişken Mühendisliği 

### Count Vectors
Kelime vektörleri oluşturacağız ve eğer metin verileri bu kelime kelimesini içeriyorsa, bu boyuta bir tane koyacağız. Bu kelimeyle her karşılaştığımızda sayıyı artıracağız, kelimeyi bir kez bile bulamadığımız her yerde 0'lar bırakacağız.
Bunun sonucu çok büyük vektörler olacaktır, eğer bunları gerçek metin verilerinde kullanırsak, metin verilerimizin kelime içeriğinin çok doğru sayılarını alırız. Ne yazık ki, bu herhangi bir anlamsal veya ilişkisel bilgi ile kullanım sağlamaz, ancak bu iyi çünkü bu tekniği kullanmanın amacı bu değildir.
Link: https://towardsdatascience.com/natural-language-processing-count-vectorization-with-scikit-learn-e7804269bb5e
### TF-IDF Vectors (words, characters, n-grams)
Bir Terim Sıklığı verilen bir dokümanda bir kelimenin kaç kez meydana geldiğinin (kelime torbasıyla eşanlamlı) bir sayıdır.Ters Belge Frekansı bir kelime belgelerin derleminden meydana sayısıdır. tf-idf kelimeleri ne kadar önemli olduklarına göre ağırlıklandırmak için kullanılır. Birçok belgede sık kullanılan kelimeler daha düşük bir ağırlığa sahip olurken, nadir olanlar daha yüksek bir ağırlığa sahip olacaktır.

TF(t) = (Bir t teriminin bir dökümanda gözlenme frekansı) / (dökümandaki toplam terim sayısı) 

IDF(t) = log_e(Toplam döküman sayısı / içinde t terimi olan belge sayısı)

Link: https://medium.com/@acrosson/summarize-documents-using-tf-idf-bdee8f60b71



# Count Vectors

In [156]:
vectorizer = CountVectorizer() 
vectorizer.fit(train_x)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [157]:
x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

In [158]:
vectorizer.get_feature_names()[5:8]

['01', '02', '03']

In [159]:
x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# TF-ID

TF(t) = (Bir t teriminin bir dökümanda gözlenme frekansı) / (dökümandaki toplam terim sayısı)

IDF(t) = log_e(Toplam döküman sayısı / içinde t terimi olan belge sayısı)

## WordLevel

In [160]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [161]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

## N-Gram

In [162]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range = (2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [163]:
x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

# CharLevel

In [164]:
tf_idf_chars_vectorizer = TfidfVectorizer(analyzer = "char", ngram_range = (2,3))
tf_idf_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [165]:
x_train_tf_idf_chars = tf_idf_chars_vectorizer.transform(train_x)
x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

# Makine Öğrenmesi ile Sentiment Sınıflandırması

# Lojistik Regrasyon scikit-learn kütüphanesi


Duyarlılık Analizi, belirli bir şeye göre birinin hissiyatını yargılamak veya duygularını anlamlandırmak için kullanılan bir yöntemdir. Temel olarak bir metin işleme (text processing) işlemi olup verilen metinin duygusal olarak
ifade etmek istediği sınıfı belirlemeyi amaçlar.
- Metinlerin üzerinden amaca yönelik olarak fikir çıkarımı yapılırken kelime sayısı, isim, sıfat, zarf veya fiil gibi kelimelerin sıklıkları (frekansları) üzerinden fikir madenciliği yapılmasına verilen isimdir. (Word2vec , TF/ IDF)
- Frekans tabanlı fikir madenciliğinde, öncelikle isim kelime grupları bulunarak bunlar uzunluklarına, kullanımdaki gerekliliklerine ve olumlu olumsuz kutupsallığına göre sınıflandırılmaktadır.

### https://medium.com/operations-management-t%C3%BCrkiye/lojistik-regresyon-ile-duygu-analizi-d9d0b8e7b4e5
### https://www.nltk.org/howto/sentiment.html
### https://www.nltk.org/api/nltk.sentiment.html#module-nltk.sentiment
### https://nlpforhackers.io/sentiment-analysis-intro/  

## Lojistik Regrasyon

In [166]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model, x_test_count, test_y, cv = 10).mean()
accuracy=float(accuracy)
print("Count Vectors Doğruluk Oranı:", accuracy)

loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(loj_model, x_test_tf_idf_word, test_y, cv = 10).mean()
print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(loj_model, x_test_tf_idf_ngram, test_y, cv = 10).mean()
print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(loj_model, x_test_tf_idf_chars, test_y, cv = 10).mean()
print("CHARLEVEL Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8538
Word-Level TF-IDF Doğruluk Oranı: 0.86
N-GRAM TF-IDF Doğruluk Oranı: 0.8176
CHARLEVEL Doğruluk Oranı: 0.9568000000000001


## Navy Bayes

In [27]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(nb_model, x_test_count, test_y, cv = 10).mean()
print("Count Vectors Doğruluk Oranı:", accuracy)

nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(nb_model, x_test_tf_idf_word, test_y, cv = 10).mean()
print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(nb_model, x_test_tf_idf_ngram, test_y, cv = 10).mean()
print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(nb_model, x_test_tf_idf_chars, test_y, cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)



Count Vectors Doğruluk Oranı: 0.865
Word-Level TF-IDF Doğruluk Oranı: 0.8744
N-GRAM TF-IDF Doğruluk Oranı: 0.8366
CHARLEVEL Doğruluk Oranı: 0.8914


## Random Forests

In [187]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(rf_model, x_test_count, test_y,cv = 10).mean()
print("Count Vectors Doğruluk Oranı:", accuracy)

rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(rf_model, x_test_tf_idf_word, test_y, cv = 10).mean()
print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(rf_model, x_test_tf_idf_ngram, test_y, cv = 10).mean()
print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(rf_model, x_test_tf_idf_chars, test_y, cv = 10).mean()
print("CHARLEVEL Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8366
Word-Level TF-IDF Doğruluk Oranı: 0.8290000000000001
N-GRAM TF-IDF Doğruluk Oranı: 0.7792000000000001
CHARLEVEL Doğruluk Oranı: 0.9974000000000001


In [61]:
?RandomForestClassifier()

Object `RandomForestClassifier()` not found.


## XGBoost

In [30]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(xgb_model, x_test_count, test_y, cv = 10).mean()
print("Count Vectors Doğruluk Oranı:", accuracy)

xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(xgb_model, x_test_tf_idf_word, test_y, cv = 10).mean()
print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(xgb_model, x_test_tf_idf_ngram, test_y, cv = 10).mean()
print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(xgb_model, x_test_tf_idf_chars, test_y, cv = 10).mean()
print("CHARLEVEL Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.7830000000000001
Word-Level TF-IDF Doğruluk Oranı: 0.7824
N-GRAM TF-IDF Doğruluk Oranı: 0.6902
CHARLEVEL Doğruluk Oranı: 1.0


# Testing

In [204]:
df=pd.read_csv('ArelUnuversitesi.csv')

In [205]:
df['sentiment']=2

In [206]:
df

,text,sentiment
0,radyosivrisinek kafaradyo radyoland yılönceben...,2
1,bbsimsek arel üniversitesi metoroloji mühendis...,2
2,ogrencidekani çocukların yeri eğitim öğretim y...,2
3,ogrencidekani çocukların yeri eğitim öğretim y...,2
4,çocukların yeri eğitim öğretim yuvaları parkla...,2
...,...,...
4398,şimdi bardağımızı önümüze alıyoruz süslemek is...,2
4399,kavanozumuzun kapağını kapattığımızdan emin ol...,2
4400,ilk olarak çay kaşığı granül kahvemizi kavanoz...,2
4401,malzemeler küçük boy kavanoz süt granül kahvet...,2


In [98]:
df['id'].iloc[7672:10003].count()

2331

In [207]:
rf = TfidfVectorizer(analyzer = "char", ngram_range = (2,3))
rf.fit(train_x)
     
dosya=input('Analiz edilen verinin kayıt edileceği csv dosyaya bir ad veriniz :')

for x in range(df['text'].count()):
    baslangic=time.time()
    
    veri= pd.Series(df['text'][x])
    veri = rf.transform(veri)

    sonuc=rf_model.predict(veri)
   
    sonuc=int(sonuc)
    
    if sonuc == 1 :
        df['sentiment'][x]=sonuc
    else:
        df['sentiment'][x]=sonuc
df.to_csv(dosya+'.csv', mode='w',index=False)
bitis=time.time()
print('İşlem tamamlandı')
print('{} dakikada toplam {} tweet analiz edildi'.format(round((bitis-baslangic)/60,2),sayi))

Analiz edilen verinin kayıt edileceği csv dosyaya bir ad veriniz : ArelUnuversitesi


İşlem tamamlandı
0.0 dakikada toplam 2331 tweet analiz edildi


In [208]:
df

,text,sentiment
0,radyosivrisinek kafaradyo radyoland yılönceben...,1
1,bbsimsek arel üniversitesi metoroloji mühendis...,1
2,ogrencidekani çocukların yeri eğitim öğretim y...,0
3,ogrencidekani çocukların yeri eğitim öğretim y...,0
4,çocukların yeri eğitim öğretim yuvaları parkla...,1
...,...,...
4398,şimdi bardağımızı önümüze alıyoruz süslemek is...,1
4399,kavanozumuzun kapağını kapattığımızdan emin ol...,1
4400,ilk olarak çay kaşığı granül kahvemizi kavanoz...,1
4401,malzemeler küçük boy kavanoz süt granül kahvet...,1


In [202]:
df.sort_values(by='sentiment',ascending=False)

,id,profile_image_url,text,created_at,timeline,retweeted,retweet_count,source,user_screen_name,user_followers_count,user_location,Hashtags,screen_name,location,statuses_count,friends_count,favourites_count,sentiment
0,1271463341144276998,http://pbs.twimg.com/profile_images/1077402606...,@RadyoSivrisinek @kafaradyo @radyoland #10YılÖ...,2020-06-12 15:24:17,<bound method User.timeline of User(_api=<twee...,False,0,Twitter for Android,SevdaAkcasu,1102,Ankara'da,"[{'text': '10YılÖnceBen', 'indices': [39, 52]}]",SevdaAkcasu,Ankara'da,8065,1082,12161,1
59,1270464754205831170,http://pbs.twimg.com/profile_images/1270807390...,SARI PAPATYA GİTMEMEN LAZIMDI \n.\n.\n.\n#surv...,2020-06-09 21:16:16,<bound method User.timeline of User(_api=<twee...,False,0,Twitter for iPhone,arel_ern,224,NaN,"[{'text': 'survivor2020', 'indices': [37, 50]}]",arel_ern,NaN,1658,267,5312,1
70,1270325535890182145,http://pbs.twimg.com/profile_images/1267557561...,🦄|-Hayatınızın yaklaşık 6’da 1’i çarşamba günl...,2020-06-09 12:03:03,<bound method User.timeline of User(_api=<twee...,False,0,Twitter for iPhone,selfcare__arel_,4,NaN,[],selfcare__arel_,NaN,178,3,5,1
69,1270326827748405250,http://pbs.twimg.com/profile_images/1267557561...,🦄| Daha fazla bu tarz gönderi gelmesi için beğ...,2020-06-09 12:08:11,<bound method User.timeline of User(_api=<twee...,False,0,Twitter for iPhone,selfcare__arel_,4,NaN,[],selfcare__arel_,NaN,178,3,5,1
67,1270394791604477955,http://pbs.twimg.com/profile_images/1269030619...,@zalimmgeceler @izellll_ Yaptım zaten Arel üni...,2020-06-09 16:38:15,<bound method User.timeline of User(_api=<twee...,False,0,Twitter for Android,fuurkanates,52558,Bursa,[],fuurkanates,Bursa,28710,35807,39586,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,1270630907666407424,http://pbs.twimg.com/profile_images/1242186829...,"RT @birsucann: Dünyada; 35 ülkede heykeli, 120...",2020-06-10 08:16:30,<bound method User.timeline of User(_api=<twee...,False,337,Twitter for iPhone,Hsyn_Arel_Ayse,23,"Buca, İzmir",[],Hsyn_Arel_Ayse,"Buca, İzmir",281,174,475,0
18,1271113674397233153,http://pbs.twimg.com/profile_images/1271019329...,~Onlara ihtiyacın olduğu zaman kaybolurlar \n~...,2020-06-11 16:14:50,<bound method User.timeline of User(_api=<twee...,False,1,Twitter for Android,arel_selfcare,2,NaN,[],arel_selfcare,NaN,11,2,0,0
77,1270295988129992704,http://pbs.twimg.com/profile_images/1270461971...,RT @SufleninVasisi: AREL Üniversitesi 2016 mez...,2020-06-09 10:05:39,<bound method User.timeline of User(_api=<twee...,False,7,Twitter for iPhone,kivircikcik23,482,"İstanbul, Türkiye","[{'text': 'muratyurtgül', 'indices': [103, 116]}]",kivircikcik23,"İstanbul, Türkiye",1447,312,15869,0
17,1271117765995384832,http://pbs.twimg.com/profile_images/1271019329...,💫|Kötü arkadaşlarını tespit etme https://t.co/...,2020-06-11 16:31:06,<bound method User.timeline of User(_api=<twee...,False,0,Twitter for Android,arel_selfcare,2,NaN,[],arel_selfcare,NaN,11,2,0,0


In [188]:
cümle= pd.Series('Bugün hava çok iyi değil')

In [189]:
rf = TfidfVectorizer(analyzer = "char", ngram_range = (2,3))
rf.fit(train_x)

veri = rf.transform(cümle)

sonuc=rf_model.predict(veri)


In [191]:
tf = TfidfVectorizer(analyzer = "char", ngram_range = (2,3))
tf.fit(train_x)

veri = tf.transform(cümle)
sonuc=loj_model.predict(veri) 
    

In [192]:
sonuc

array([1])

In [213]:
df.groupby('sentiment').count()

,text
sentiment,
0,1452
1,2951
